In [1]:
#reference: https://blog.csdn.net/qq_40341531/article/details/105165014


from urllib.parse import quote
from urllib import request
import json
import xlwt
import pandas as pd
from tqdm import tqdm

In [2]:
amap_web_key = '3afde22baea9c968ff9f732a3a3d9891'

poi_search_url = "http://restapi.amap.com/v3/place/text"
poi_boundary_url = "https://ditu.amap.com/detail/get/detail"

cityname = '上海'
#city_areas = []
# classes = ['充电站']
'''["体育休闲服务场所", "运动场所",
"综合体育馆",
"保龄球馆",
"网球场",
"篮球场馆",
"足球场",
"滑雪场",
"溜冰场",
"户外健身场所",
"海滨浴场",
"游泳馆",
"健身中心",
"乒乓球馆",
"台球厅",
"壁球场",
"马术俱乐部",
"赛马场",
"橄榄球场",
"羽毛球场",
"跆拳道场馆",
"高尔夫相关",
"高尔夫球场",
"高尔夫练习场"]'''

'["体育休闲服务场所", "运动场所",\n"综合体育馆",\n"保龄球馆",\n"网球场",\n"篮球场馆",\n"足球场",\n"滑雪场",\n"溜冰场",\n"户外健身场所",\n"海滨浴场",\n"游泳馆",\n"健身中心",\n"乒乓球馆",\n"台球厅",\n"壁球场",\n"马术俱乐部",\n"赛马场",\n"橄榄球场",\n"羽毛球场",\n"跆拳道场馆",\n"高尔夫相关",\n"高尔夫球场",\n"高尔夫练习场"]'

## Coordinates generator

In [4]:
#!/usr/bin/env python 
# -*- coding:utf-8 -*-

#经纬度坐标（WGS84）国测局火星坐标（GCJ02）百度坐标（BD09）相互转换
'''
目前的的坐标体系分类：
一是GPS坐标，也即WGS-84坐标是一个国际的标准，一般卫星导航，原始的GPS设备中的数据都是采用这一坐标系。国外的Google地图、OSM等采用的都是这一坐标。
二是国测局坐标，国测局坐标GCJ-02坐标也叫火星坐标，是国家测绘局为了国家安全在原始坐标的基础上进行偏移得到的坐标，基本国内的电子地图、导航设备都是采用的这一坐标系，如：高德、搜搜、51地图MapABC地图，谷歌中国地图也是。
三是百度坐标，百度坐标BD-09坐标是百度公司出于商业保护在国测局坐标基础上进行的二次加密。
'''

import json
import requests
import math

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方


class Geocoding:
    def __init__(self, api_key):
        self.api_key = api_key

    def geocode(self, address):
        """
        利用高德geocoding服务解析地址获取位置坐标
        :param address:需要解析的地址
        :return:
        """
        geocoding = {'key': self.api_key,
                     'city': cityname,
                     'address': address}
        # geocoding = urllib.urlencode(geocoding)
        # ret = urllib.urlopen("http://restapi.amap.com/v3/geocode/geo{}".format(geocoding))
        url = "http://restapi.amap.com/v3/geocode/geo?"
        ret = requests.get(url, params=geocoding)
        #print(ret.json())
        if ret.status_code == 200:
            # res = ret.json()
            # json_obj = json.loads(res)
            json_obj = ret.json()
            if json_obj['status'] == '1' and int(json_obj['count']) >= 1:
                geocodes = json_obj['geocodes'][0]
                lng = float(geocodes.get('location').split(',')[0])
                lat = float(geocodes.get('location').split(',')[1])
                return [lng, lat]
            else:
                return None
        else:
            return None


def gcj02_to_bd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]


def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]


def wgs84_to_gcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]


def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]


def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)


def wgs84_to_bd09(lon, lat):
    lon, lat = wgs84_to_gcj02(lon, lat)
    return gcj02_to_bd09(lon, lat)


def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret


def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

## Scraping POI data

In [5]:
# 单页获取pois
def getpoi_page(cityname, types, page):
    req_url = poi_search_url + "?key=" + amap_web_key + '&extensions=all&types=' + quote(
        types) + '&city=' + quote(cityname) + '&citylimit=true' + '&offset=25' + '&page=' + str(
        page) + '&output=json'
    data = ''
    with request.urlopen(req_url) as f:
        data = f.read()
        data = data.decode('utf-8')
    return data

In [6]:
# 将返回的poi数据装入集合返回
def hand(poilist, result):
    # result = json.loads(result)  # 将字符串转换为json
    pois = result['pois']
    for i in range(len(pois)):
        poilist.append(pois[i])

In [7]:
# 根据城市名称和分类关键字获取poi数据
def getpois(cityname, keywords):
    i = 1
    poilist = []
    while True:  
        result = getpoi_page(cityname, keywords, i)
        result = json.loads(result)  # 将字符串转换为json
        if result['count'] == '0':
            break
        hand(poilist, result)
        i = i + 1
    return poilist

In [8]:
def write_to_excel(poilist, cityname, classfield):
    # 一个Workbook对象，这就相当于创建了一个Excel文件
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet(classfield, cell_overwrite_ok=True)

    # 第一行(列标题)
    sheet.write(0, 0, 'x')
    sheet.write(0, 1, 'y')
    sheet.write(0, 2, 'count')
    sheet.write(0, 3, 'name')
    sheet.write(0, 4, 'adname')
    sheet.write(0, 5, 'Category-1')
    sheet.write(0, 6, 'Category-2')
    sheet.write(0, 7, 'Category-3')
    #sheet.write(0, 8, 'address')


    for i in range(len(poilist)):
        location = poilist[i]['location']
        name = poilist[i]['name']
        #address = poilist[i]['address']
        adname = poilist[i]['adname']
        lng, lat = str(location).split(",")
        c1,c2,c3 = poilist[i]['type'].split(';')[:3]
        

        #坐标转换
        result = gcj02_to_wgs84(float(lng), float(lat))
        lng = result[0]
        lat = result[1]


        # 每一行写入
        sheet.write(i + 1, 0, lng)
        sheet.write(i + 1, 1, lat)
        sheet.write(i + 1, 2, 1)
        sheet.write(i + 1, 3, name)
        sheet.write(i + 1, 4, adname)
        sheet.write(i + 1, 5, c1)
        sheet.write(i + 1, 6, c2)
        sheet.write(i + 1, 7, c3)
        #sheet.write(i + 1, 8, address)

    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + cityname + "_" + classfield + '.xls')

## 充电站数据

In [8]:
classes_all_pois = []
for clas in classes:
    pois_area = getpois(cityname, clas)
    classes_all_pois.extend(pois_area)
print("数据总数为：" + str(len(classes_all_pois)))
write_to_excel(classes_all_pois, cityname, )
print('================分类：'  + str(clas) + "写入成功")


{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":"B0GK27JKVR","distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":"1","url":"http://store.is.autonavi.com/showpic/3eeec0b020d68e6d3ef795e35b87333e"},{"title":"1","url":"http://store.is.autonavi.com/showpic/7e906a3b1d280fce78ca1733f4cb58aa"},{"title":"1","url":"http://store.is.autonavi.com/showpic/ef49b2075ac93e154734888600ccd1c5"}],"discount_num":"0","gridcode":"4621730621","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"静安区","children":[],"alias":[],"tel":"4001300001","id":"B0FFLFSATO","tag":[],"event":[],"entr_location":"121.458221,31.256314","indoor_map":"0","email":[],"timestamp":"2022-03-31 19:01:44","website":[],"address":"芷江西路488号","adcode":"310106","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"和田","indoor_data":{"cmsid":[],"truefloor

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":"B0H0X5604G","distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/ce02297d2d15bc38db0006e0e14d824f"},{"title":[],"url":"http://store.is.autonavi.com/showpic/80cce6d22a065799e331587b7a4b2edc"},{"title":[],"url":"http://store.is.autonavi.com/showpic/401bdc28792a583f8f0b65fe7e7b63eb"}],"discount_num":"0","gridcode":"4621712722","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"嘉定区","children":[],"alias":[],"tel":"4008280768","id":"B0HKO1LDJJ","tag":[],"event":[],"entr_location":[],"indoor_map":"0","email":[],"timestamp":"2022-03-31 19:42:16","website":[],"address":"曹安公路4589号","adcode":"310114","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"黄渡","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor"

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[],"discount_num":"0","gridcode":"4721030710","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"宝山区","children":[],"alias":[],"tel":"021-59923613","id":"B0G2A549IF","tag":[],"event":[],"entr_location":"121.463485,31.336784","indoor_map":"0","email":[],"timestamp":"2022-03-31 19:43:16","website":[],"address":"联长路140号","adcode":"310113","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"泗塘","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor":[]},"childtype":[],"exit_location":[],"name":"瑜珩汽车充电站(示范站宝山联长路站)","location":"121.463502,31.336684","shopid":[],"navi_poiid":[],"groupbuy_num":"0"},{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","ty

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[],"discount_num":"0","gridcode":"4621639702","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"静安区","children":[],"alias":[],"tel":[],"id":"B0FFHY1DRN","tag":[],"event":[],"entr_location":"121.471498,31.243371","indoor_map":"0","email":[],"timestamp":"2022-03-31 14:26:13","website":[],"address":"西藏北路166号大悦城地下停车场(B3北区)","adcode":"310106","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"七浦路","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor":[]},"childtype":[],"exit_location":[],"name":"充电网汽车充电站","location":"121.471851,31.243298","shopid":[],"navi_poiid":"H51F010012_1270865","groupbuy_num":"0"},{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recomm

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":"1","url":"http://store.is.autonavi.com/showpic/a091a02fdb7467941a7275986bc29584"},{"title":"1","url":"http://store.is.autonavi.com/showpic/588751689dac8823e4f3853b5dcb5bf2"},{"title":"1","url":"http://store.is.autonavi.com/showpic/0f2613805157e2d64b47ac238611d742"}],"discount_num":"0","gridcode":"4621519401","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"青浦区","children":[],"alias":[],"tel":"4001300001","id":"B0H3G43T3G","tag":[],"event":[],"entr_location":"121.181589,31.159484","indoor_map":"0","email":[],"timestamp":"2022-03-31 19:01:03","website":[],"address":"盈港东路6433号","adcode":"310118","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":[],"indoor_data":{"cmsid":[],"truefloor":[],"cpid"

...

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":"B0H6MHGLLR","distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":"1","url":"http://store.is.autonavi.com/showpic/9027e750598e73e2138707a7fb7c7ac9"},{"title":"1","url":"http://store.is.autonavi.com/showpic/889f82088d9a08b2889d652fd0a90c60"},{"title":"1","url":"http://store.is.autonavi.com/showpic/1aa316fe1b5360c3cfc58207fc3cb268"}],"discount_num":"0","gridcode":"4621649001","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"浦东新区","children":[],"alias":[],"tel":"4001300001","id":"B0FFHZZHJ3","tag":[],"event":[],"entr_location":"121.506163,31.242782","indoor_map":"0","email":[],"timestamp":"2022-03-31 19:01:45","website":[],"address":"陆家嘴环路1088号","adcode":"310115","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"陆家嘴","indoor_data":{"cmsid":[],"truef

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[],"discount_num":"0","gridcode":"4621637021","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"普陀区","children":[],"alias":[],"tel":[],"id":"B0G0MRNQPI","tag":[],"event":[],"entr_location":[],"indoor_map":"0","email":[],"timestamp":"2022-03-12 15:30:55","website":[],"address":"真北路818号","adcode":"310107","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"金沙江路","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor":[]},"childtype":[],"exit_location":[],"name":"汽车充电站(上海近铁城市广场南座&北座)","location":"121.381915,31.232625","shopid":[],"navi_poiid":[],"groupbuy_num":"0"},{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/4abc924eb6ba62fc18b061bff861fc3e"},{"title":[],"url":"http://store.is.autonavi.com/showpic/2fe4d8793b6d0720ae9c653c29e81005"},{"title":[],"url":"http://store.is.autonavi.com/showpic/4f54c2fc4b05421ef5ec5029a8ac4c36"}],"discount_num":"0","gridcode":"4621545012","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"浦东新区","children":[],"alias":[],"tel":[],"id":"B0FFGZHEZQ","tag":[],"event":[],"entr_location":[],"indoor_map":"0","email":[],"timestamp":"2022-03-18 23:05:12","website":[],"address":"林德路与三新路交叉口西南80米","adcode":"310115","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"三林镇","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor":[]},"childt

{"suggestion":{"keywords":[],"cities":[]},"count":"891","infocode":"10000","pois":[{"parent":[],"distance":[],"pcode":"310000","importance":[],"biz_ext":{"cost":[],"rating":[]},"recommend":"0","type":"汽车服务;充电站;充电站","photos":[{"title":[],"url":"http://store.is.autonavi.com/showpic/e5fdc4ce2dfbc4cc0fdb50bb219a9207"},{"title":[],"url":"http://store.is.autonavi.com/showpic/66c246343501f590fc8d7ad5988d92e3"},{"title":[],"url":"http://store.is.autonavi.com/showpic/6b934601c60e787bb74a2b90fb16e476"}],"discount_num":"0","gridcode":"4621437302","typecode":"011100","shopinfo":"0","poiweight":[],"citycode":"021","adname":"闵行区","children":[],"alias":[],"tel":[],"id":"B0FFIZY613","tag":[],"event":[],"entr_location":[],"indoor_map":"0","email":[],"timestamp":"2022-03-17 09:27:57","website":[],"address":"元江路与都会路交叉口西北150米","adcode":"310112","pname":"上海市","biz_type":[],"cityname":"上海市","postcode":[],"match":"0","business_area":"颛桥","indoor_data":{"cmsid":[],"truefloor":[],"cpid":[],"floor":[]},"childty

{"suggestion":{"keywords":[],"cities":[]},"count":"0","infocode":"10000","pois":[],"status":"1","info":"OK"}
数据总数为：891
================分类：充电站写入成功


In [ ]:
geocoding = Geocoding(amap_web_key)
data = pandas.read_excel('charging station.xlsx')

location_list = []

not_found = 0

cnt = 0

for i in data['Address']:
    cnt += 1
    if cnt % 1000 == 0:
        print(cnt / 8311)
    if geocoding.geocode(i) is None:
        location_list.append("Not Found")
        not_found += 1
        continue
    lng, lat = geocoding.geocode(i)
    location_list.append(gcj02_to_wgs84(float(lng), float(lat)))

In [36]:
for i in location_list:
    if i is None:
        print(None)
        continue
    print(i[0], i[1])    

0.12032246420406689
0.24064492840813378
0.3609673926122007
0.48128985681626757
0.6016123210203345
0.7219347852244014
0.8422572494284682
0.9625797136325351
[[121.47240240725365, 31.233961382700485], [121.4703395124808, 31.23493224092388], [121.47136783307937, 31.235661734384436], [121.47136783307937, 31.235661734384436], [121.47113300564034, 31.236276804426215], [121.47253664992121, 31.236345660409942], [121.48007509331917, 31.238996894849883], [121.47308846982142, 31.236585089049093], [121.47307573980919, 31.232931733721994], [121.46917857632037, 31.23665728135191], [121.46917794260735, 31.232358257753024], [121.47594319809745, 31.235368690324492], [121.47434200975478, 31.232671404305247], [121.46917794260735, 31.232358257753024], [121.46636969545204, 31.21168445763462], [121.46917794260735, 31.232358257753024], [121.47014064642106, 31.237439546488595], [121.4651570990157, 31.23230653495458], [121.4704786559342, 31.23858494925879], [121.47214984490694, 31.230725176601144], [121.4740722

## 公共运动设施

In [5]:
geocoding = Geocoding(amap_web_key)
data = pandas.read_excel('Public Sports Facilities.xlsx')

location_list = []

not_found = 0

cnt = 0

for i in data['Address']:
    cnt += 1
    if cnt % 1000 == 0:
        print(cnt / data.shape[0])
    if geocoding.geocode(i) is None:
        location_list.append("Not Found")
        not_found += 1
        continue
    lng, lat = geocoding.geocode(i)
    location_list.append(gcj02_to_wgs84(float(lng), float(lat)))

0.05595344673231871
0.11190689346463742
0.16786034019695614
0.22381378692927484
0.27976723366159356
0.3357206803939123
0.39167412712623095
0.4476275738585497
0.5035810205908684
0.5595344673231871
0.6154879140555058
0.6714413607878246
0.7273948075201433
0.7833482542524619
0.8393017009847806
0.8952551477170994
0.9512085944494181


In [12]:
for i in location_list:
    if type(i[0]) != float:
        print('N/A', 'N/A')
        continue
    print(i[0], i[1])

121.46569407866076 31.287156039824406
121.46235284859436 31.29696240082344
121.45758183456034 31.28291220247118
121.46299600036986 31.283271600495354
121.46230479264517 31.295905699276993
121.46199152398995 31.299534457950916
121.46150301533264 31.299810107407268
121.46563376739294 31.300191645284436
121.46362754703341 31.287957309468997
121.4661873861643 31.301734501563256
121.45830253226072 31.287580043244823
121.4643310585002 31.28417468671692
121.45909028108348 31.28255604317479
121.46296400055867 31.28212499180034
121.46299600036986 31.283271600495354
121.45812696982887 31.288150381419054
121.46199152398995 31.299534457950916
121.46500184931742 31.287745977610275
121.4694401466181 31.28657032460583
121.46616447089217 31.299904162060777
121.46407432934701 31.29498163141408
121.4908681153029 31.264644373898957
121.4979999168904 31.274951343995582
121.49680056292884 31.27531659649619
121.49442720235598 31.273040442302793
121.49487167560292 31.274550137083732
121.62791482740698 31.541

121.36640186943507 31.105053412257742
121.69110516451815 31.56485884470362
121.44198930183676 31.6146330737375
121.40234878707275 31.12210144315043
121.56728808239886 31.155030695514863
121.6582457721644 31.617258786562033
121.44198930183676 31.6146330737375
121.69495433889165 31.57057700870184
121.69489445071532 31.570391119189892
121.40735341648735 31.137421745683046
121.1366416901777 30.872152469357133
121.64419898107826 31.60148350986009
121.63108203370156 31.607732828576342
121.70582681582677 31.56722483294846
121.64986101986692 31.57640775953123
121.39115688247091 31.136733002632855
121.39115688247091 31.136733002632855
121.39099089409245 31.137217790523227
121.39004937949183 31.135443035972912
121.49097238332588 31.206994126177246
121.1039524936267 31.147602035216853
121.40246145656135 31.12624621212715
121.40399290453021 31.12647446132025
121.40576756019209 31.12695776289383
121.4103380909211 31.127392631021333
121.40622901019142 31.12418044090185
121.40799110369865 31.12197038

121.05638098196812 31.111560314308367
121.22550028523668 31.23025232524478
121.1791049019146 31.196073699111963
121.17345000142245 31.202550060599435
121.13912216611578 31.22988339773289
121.05980397487149 31.113454871633746
121.33623949916054 31.056849640818076
121.16306543478103 30.901589259153344
121.37425868119773 30.852749040921633
121.33536761210792 30.85602926619829
121.28866012550462 30.779695928309856
121.1343274431731 31.06695463158645
121.24009050306134 30.83717935651624
121.24009050306134 30.83717935651624
121.24779856228032 30.81963463039274
121.24779856228032 30.81963463039274
121.01475922156764 30.895156658069194
120.9892758138326 30.89906507319161
121.15422128778107 30.895851265046876
121.03627627348148 30.858698461621277
121.05454286506522 30.854279232897966
121.09953837794839 30.880549562960596
121.081928826001 30.882683661874815
121.75413058201597 31.17430471607816
121.33879159646096 30.862541186986846
121.30681665589883 30.72542625471062
121.31398481592265 30.726202

121.17835399259543 30.834492663384275
121.51188047953431 31.616007777594966
121.57769652636242 30.91006623115396
N/A N/A
121.4668830080217 31.004583069255855
121.71376803966827 31.515417197288283
121.04775546379672 30.949982708039016
121.45024330842453 31.111321113776523
121.37401838221218 31.04480635164718
121.4823108737886 31.215772244232483
121.48254667355457 31.216089783116804
121.48362453408947 31.215245348521673
121.48362453408947 31.215245348521673
121.48255781090943 31.214935366509124
121.48265220659108 31.214178004980546
121.47813640744013 31.215367326444028
121.47947394989797 31.216233769721626
121.4765479452887 31.215607664843375
121.48950429063781 31.216810028088293
121.48950429063781 31.216810028088293
121.48530032633205 31.216729580212572
121.48421143461833 31.217226131621217
121.48538950889467 31.218297098606943
121.48901425122745 31.22018696448436
121.49289889527375 31.223597966497746
121.4873742730077 31.221667385160433
121.48426545039483 31.21928131507605
121.48684258

121.30156870330002 31.442173340435122
121.2483801199327 31.40417532563188
121.2483801199327 31.40417532563188
121.24062457075479 31.406156589063368
121.23146489659793 31.371538593595076
121.15661485717136 30.98000387064426
121.03570588649815 30.930284908718495
120.94468265048533 31.065638301856467
120.94032576904644 31.118501160967156
120.92090638811547 31.11755715059525
120.9208543920978 31.11836176454279
120.88506149845203 31.130891752882015
121.25914810843665 31.3316898859095
121.52175995009351 30.964764562206494
121.54246660083672 31.296684078669458
121.4262207282084 31.28746822068758
121.42685885499422 31.285794723102583
121.38679196564784 31.111258501513596
121.5722431270065 31.130608942242056
121.54101376381817 31.29585425709441
121.4553032197274 31.307171041629214
121.59899208414949 31.123446175647885
121.38777245144465 31.3521251067952
121.38386516998196 31.358523951201324
121.37983130142665 31.35832769587031
121.38680845171443 31.3577536590366
121.3960325905255 31.34685082296

...

121.47520635204819 31.29784679460184
121.44011223403167 31.303615403667756
121.42470540665327 31.242371779915533
121.47745501730236 31.29787318573568
121.47593396436308 31.272823125459716
121.48211171015015 31.29159918104359
121.47711806831163 31.29546618074411
121.42200149353671 31.24468564234141
121.42200149353671 31.24468564234141
121.48551289283334 31.289389078293222
121.47682124484062 31.293220233321026
121.46910670608456 31.290715731237736
121.49716349530075 31.291540992873394
121.49365554935474 31.295103046063637
121.46220892139824 31.256142146230648
121.42135059329847 31.256902511151353
121.4908434440369 31.29501966288812
121.463074522027 31.25676209387665
121.42333952752577 31.253369255085428
121.4888324915007 31.292044857369145
121.47342961263851 31.3066223498683
121.42379469762031 31.293104987514525
121.46205923523979 31.258811593992505
121.48919987018154 31.2912892573432
121.47591601850449 31.30652040109702
121.48807893708845 31.290420270828438
121.48924908161494 31.2908745

121.47022135584238 31.208877199704194
121.47011101137436 31.208682991452864
121.47029867383753 31.20811977736863
121.47126683043304 31.20842329250375
121.4869890481551 31.2081736860596
121.45353443751598 31.22430551399442
121.47228822508183 31.208269194817554
121.47235647588228 31.207988520154842
121.46892839927924 31.20589312076774
121.47179061126376 31.207841021642054
121.46099596247939 31.224498454500036
121.47393882844946 31.20722431082222
121.46949220909316 31.206146529948427
121.46761304479345 31.207217930501574
121.46872069627916 31.20624738616704
121.45443418480629 31.22135199085486
121.46391353210474 31.205326078392563
121.45464281216245 31.220828725382226
121.46774446448693 31.20724127201225
121.45396093569205 31.220588250795597
121.4661947516977 31.204555439517996
121.46844462579001 31.208327632580176
121.45731731720932 31.21035208767449
121.46424444344112 31.206690277174587
121.45731731720932 31.21035208767449
121.46087983569592 31.210263929044494
121.47592845979456 31.2082

121.8358079596099 31.02921816691926
121.55467205899727 31.01823236750585
121.10058275505905 30.901408641139763
121.08365312756628 30.89950897281086
121.53855171737055 30.876498854407895
121.74119144897723 31.148291445913102
121.12128096140653 31.0338970131673
121.42624981576654 30.975107696797306
121.22962243763186 31.10587282779179
121.47790238140486 31.253104285743927
121.88298010543541 31.009343949989294
121.43778999284147 31.29514859353574
121.5132649288885 31.11158381994967
121.70115645741183 30.937233120914577
121.73532978376363 30.930859152995776
121.73349487649564 30.93363631091229
121.41511900089513 31.27647838890516
121.38564714997138 31.323000399331118
121.44861064690974 30.91790112687266
121.49589126468231 30.919616175858756
121.45643597502684 30.915976040114277
121.47423920787313 30.920427091634895
121.44955076287492 30.933149138568524
121.36933707362451 30.952171810511913
121.44955823216709 30.904438109664724
121.44877240952984 30.902088791824543
121.39672193561526 30.905

121.36489774450945 31.258204386407947
121.19474418944357 31.37177587666859
121.19474418944357 31.37177587666859
N/A N/A
121.46581799200483 31.246154964752368
121.3239093006612 31.28842508245242
121.1460754169479 31.39174416017193
121.15198015708826 31.37302811753105
121.14463366358471 31.346969875205097
121.16298099042153 31.36228177307107
121.14327430155664 31.346183876708935
121.15198015708826 31.37302811753105
121.15198015708826 31.37302811753105
121.65404633628488 31.40932969509151
121.24412777897659 31.345508687692874
121.29556610103431 31.381476522743306
N/A N/A
N/A N/A
N/A N/A
121.3144147106796 31.371903022833408
N/A N/A
121.57665335646575 30.947761158947273
N/A N/A
121.27169324050215 31.36365155582299
121.31429449087092 31.354433911081173
121.29175198853616 31.339716202681203
121.1680006022441 31.392911818548814
121.32713969336999 31.337391229489874
121.29556610103431 31.381476522743306
121.3135400663045 31.332689873365037
121.28132261433208 31.38976974117833
121.25683945627573

121.22167678440431 31.388478714356786
121.22473602447643 31.37977678020077
121.22111257557366 31.385050239220284
121.2175983845769 31.38268966105265
121.21552924654101 31.381823801014995
121.21605951211446 31.38553424629682
121.21224726048362 31.38374858530835
121.21439395885277 31.370183172905666
121.21666738522512 31.371223249675708
121.21320946583549 31.371473450611678
121.21431668708962 31.37233241288189
121.22236845698801 31.375848105159317
121.42274232200454 31.165847699382223
121.23632516828032 31.40168972944554
121.45287154252156 31.190380408925048
121.21043987925071 31.34991325243907
121.21043987925071 31.34991325243907
121.40759630260538 31.243621123532453
121.22259127847073 31.380157970109163
121.24257452811824 31.395988684875338
121.23085185675339 31.391946112244867
121.25348801389774 31.40729935528373
121.20203294524299 31.373812321809616


## Geocoding

In [11]:
from tqdm import tqdm

In [13]:
geocoding = Geocoding(amap_web_key)
data = pd.read_excel('房天下-房价.xlsx')

lng_list = []
lat_list = []

not_found = 0

cnt = 0

for i in tqdm(data['地址']):
    cnt += 1
    if geocoding.geocode(i) is None:
        lng_list.append("nan")
        lat_list.append("nan")
        not_found += 1
        continue
    lng, lat = geocoding.geocode(i)
    lng, lat = gcj02_to_wgs84(float(lng), float(lat))[0], gcj02_to_wgs84(float(lng), float(lat))[1]
    lng_list.append(lng)
    lat_list.append(lat)

100%|██████████| 13892/13892 [21:39<00:00, 10.69it/s]


In [14]:
data['lng'] = lng_list
data['lat'] = lat_list

In [16]:
data.to_excel('geocoded_neighborhoods.xlsx')

## Specific POIs

In [9]:
classes = ['住宅区']
types = ['120300', '150900', '050000', '090100', '100100', '060000']
for typ in tqdm(types):
    classes_all_pois = []
    pois_area = getpois(cityname, typ)
    classes_all_pois.extend(pois_area)
    print("数据总数为：" + str(len(classes_all_pois)))
    write_to_excel(classes_all_pois, cityname, typ)
    print('================分类：'  + str(typ) + "写入成功")


 17%|█▋        | 1/6 [00:02<00:14,  2.97s/it]

数据总数为：200
================分类：120300写入成功


 33%|███▎      | 2/6 [00:04<00:09,  2.36s/it]

数据总数为：187
================分类：150900写入成功


 50%|█████     | 3/6 [00:07<00:07,  2.48s/it]

...

数据总数为：186
================分类：090100写入成功


 83%|████████▎ | 5/6 [00:12<00:02,  2.61s/it]

数据总数为：200
================分类：100100写入成功


100%|██████████| 6/6 [00:15<00:00,  2.62s/it]

数据总数为：198
================分类：060000写入成功
